# Genre prediction function

Here we construct the function performing genre prediction based on input lyrics, allowing user interaction. The function combines our two best models: Vectorization measured on logistic regression and sequentialisation, measured on a Multiple Layer Perceptron (MLP), Feed Forward Neural Network (FFNN).

We first import the necessary libraries.

### 0. Requirements

The following modules are required for our main algorithm:

In [1]:
## Standard libraries for data
import numpy as np
import pandas as pd

## Libraries for visualisation
import seaborn as sns
import matplotlib.pyplot as plt

## Standard libraries for NLP
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize

## Tokenizers and stemmers
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer, SnowballStemmer
## nltk.download('stopwords')
## nltk.download('punkt')

## Vectorizer and LogReg for model predictions
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

## Model tuning, measuring and execution methods from sklearn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics

## Text manipulation tools
from word2number import w2n
import string
import re

## Miscellaneous
from time import time
import pickle
from scipy.spatial.distance import hamming
import random
from random import sample

In [2]:
## Set working directory accordingly
import os
os.chdir("D://Datasets//Various Datasets")

In [3]:
## Tensorflow and tensorflow modules, used for NN configuration, architecture, build-up etc
from tensorflow.python.eager.context import get_config
from tensorflow import keras 
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

## TF tools for layer processing and for NLP specifically
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

## Keras tools for model performance monitoring
# from keras import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

## Keras tools for visualisation
from keras.utils.vis_utils import plot_model
import pydot
import graphviz

The following global variables and functions are required for our main algorithm:

In [4]:
## Define the function for regex cleaning
def regex_clean(txt, regex, sub=' '):
        
    return " ".join(re.sub(regex, sub, txt).split()) ## Substitute the desired regex with nothing,
                                                    ## then bring the sentence back together

In [5]:
## Add the stopwords, as used in the main workbook
l1 = stopwords.words('english')
string.punctuation
l1.extend(string.punctuation)
irr = ['chorus', 'verse', 'verses', 'choruses']
l1.extend(irr)

In [6]:
## Define the 3 types of stemmers, as linked in the workbook
p_stemmer = PorterStemmer()
l_stemmer = LancasterStemmer()
s_stemmer = SnowballStemmer(language='english')

In [7]:
## Data preparation function, for general purpose use
def prep_data(song, stem='s'):
    
    assert(stem=='s' or stem=='l' or stem=='p'), '''Input a correct stemming parameter and try again.
    The only accepted types are s for Snowball, l for Lancaster or p for Porter. Default is Snowball.'''
    
    song = song.lower() ## Transform the sentence into lowercase
   #  review = regex_clean(review, "'")
    song = regex_clean(song, r'(\w)\1{2,}', r'\1')
    song = regex_clean(song, r"'\s|'\.", r'g')
    song = regex_clean(song, r'(\[.*?\])', r'')
    song = regex_clean(song, r'(\W){2,}', r'\1')
    
    ## Tokenize 
    song = nltk.wordpunct_tokenize(song) ## tokenize the string

    ## Post Token Cleaning - Stuff that applies to a list 
        
    song = [word for word in song if word not in l1] ## Eliminate all extended stopwords from among our tokens
    song = [globals()['{}_stemmer'.format(stem)].stem(word) for word in song] ## Apply the chosen stemmer
    song = [word for word in song if not word.isdigit()]
    song = [word for word in song if len(word)>2] ## Only consider words longer than 2 characters, to avoid 'de, tg, ll' etc.

##   song = [word for word in song if word not in l1] ## Re-remove stopwords for safety
    
    return song

### 1. Function definition + testing

Here we define the function which performs the predictions and test it using Test Driven Development (TDD):

##### 1.1 Definition 

In [8]:
## Main Genre Predictive function

def GenrePredictor(lyrics, Neural_only=False):
    
    ''' Input lyrics from a song, ouput the predicted genre of the song '''
    
    ## Prepare the song and ensure the input is valid
    assert(type(lyrics)==str), "Input lyrics as a STRING."
    lyrics_prep = prep_data(lyrics)
    L = len(lyrics_prep)
    
    ## Dictionary to match the genre back to the numeric result
    genre_matcher = {0:'Rock', 1:'Hip Hop', 2:'Pop'}
    
    
    ## LogReg component of the algorithm
    ## Executes only when less than 200 or more than 350 words in the song AND Neural is turned off
    
    if(((L<200 and L>1) or L>350) and Neural_only==False):
        
        ## Load in necessary items from main: lr model, vectorizer, keywords
        file_1 = open('logreg_model.sav', 'rb')
        file_2 = open('vector.sav', 'rb')
        lr = pickle.load(file_1)
        vector = pickle.load(file_2)
        Hop = set(['nigga', 'shit', 'bitch', 'fuck', 'hit', 'gon', 'niggaz', 'with', 'bout', 'hoe', 'cuz'])
        Rock = set(['blue', 'dead', 'line', 'woman', 'child', 'cold', 'town', 'year',
       'help', 'alive', 'behind'])
        Pop = set(['kiss', 'touch', 'christmas', 'hurt', 'beautiful', 'matter', 'somebody', 'trying', 'else', 'knew', 'forget'])
        
        ## Get the 'Trails' parameter of the song, similarly to the main workbook
        lyric_clean_trail = regex_clean(lyrics, r"'\s|'\.", r'g')
        lyric_clean_trail = nltk.wordpunct_tokenize(lyric_clean_trail)
        lyric = [nltk.wordpunct_tokenize(lyrics)]
        Trails = len(lyric) - len(lyric_clean_trail)
        if(Trails<0):
            Trails=0
        
        ## Get the rock,pop and hop parameters of the song, similarly to the main workbook
        song = set(lyrics_prep)
        Rock_lst = int([1 if len(song&Rock)>0 else 0][0])
        Pop_lst = int([1 if len(song&Pop)>0 else 0][0])
        Hop_lst = int([1 if len(song&Hop)>0 else 0][0])
        
        ## Prepare the song in vectorizable format. Vectorize it
        lyrics_prep = ' '.join(lyrics_prep)
        lyrics_prep = [lyrics_prep]
        lyric = vector.transform(lyrics_prep)
        lyric = pd.DataFrame(lyric.toarray())
        
        ## Create the test set in the same shape as expected input for the model
        X_test = pd.concat([lyric, pd.Series(Trails), pd.Series(Rock_lst), pd.Series(Pop_lst), pd.Series(Hop_lst)], axis=1)
        
        ## Close the files accessed from main
        file_1.close()
        file_2.close()
        
        ## Output predicted genre
        return genre_matcher[int(lr.predict(X_test)[0])]
    
    ## MLP component of the algorithm
    ## Executes when between 200 and 350 words in the song, OR when Neural is turned on 
    else:
        
        ## Define the Keras tokenizer
        tokenizer = Tokenizer(
        num_words=None,
        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
        lower=True, split=' ')
        
        ## Load required data for the model: lyrics themselves (for tokenizer fit), sequences and target
        file_1 = open('lyrics.sav', 'rb')
        file_2 = open('sequences.sav', 'rb')
        file_3 = open('Target.sav', 'rb')
        data = pickle.load(file_1)
        seq = pickle.load(file_2)
        genre = pickle.load(file_3)
        
        ## Fit the tokenizer, then transform our input song into sequence using it
        tokenizer.fit_on_texts(data)
        lyrics_prep = ' '.join(lyrics_prep)
        lyrics_prep = [lyrics_prep]
        test = tokenizer.texts_to_sequences(lyrics_prep)
        test = pd.Series(test)
        test = test[0]
        
        ## Set threshold for optimal performance to the song's # of words
        threshold=len(test)
        
        ## Gather the training set suited for our song: i.e. above the threshold
        test = [test]
        X = seq
        y = genre
        X_good = X[X.str.len()>threshold]
        idx = X_good.index
        y = y[y.index.isin(idx)].copy()
        X_good = X_good.reset_index(drop=True)
        y = y.reset_index(drop=True)
        
        ## Define the adequate features
        for i in range(threshold):
            globals()['X_good_{}'.format(i)] = [int(X_good[j][i]) for j in range(len(X_good))]
        for i in range(threshold):
            globals()['X_good_{}'.format(i)] = pd.Series(globals()['X_good_{}'.format(i)])
        
        ## Concatenate them into a single data, then (optional!) perform the train/test split
        try:
            X_final = pd.concat([globals()['X_good_{}'.format(i)] for i in range(threshold)], axis=1)
            X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size = 0.2, random_state = 124, stratify=y)
        except ValueError:
            file_1.close()
            file_2.close()
            file_3.close()
            return "Please input MEANINGFUL lyrics. As far as I'm concerned, that's not really a song."
        
        ## Fit the model on the data (with the chosen threshold)
        clf = MLPClassifier(random_state=1, max_iter=300, activation='logistic', learning_rate='adaptive',
                   hidden_layer_sizes=(15,10,6,3,3,6,10,15)).fit(X_train, y_train)
        ## Close the files accessed from main
        file_1.close()
        file_2.close()
        file_3.close()
        
        ## Output predicted genre
        return genre_matcher[int(clf.predict(test)[0])]     

##### 1.2 TDD

In [9]:
## Random fragments of songs, to test basic functionality of the algorithm
song_input_random_1 = '''If it hadn't been for Cotton-Eye Joe
I'd been married long time ago
Where did you come from, where did you go?
Where did you come from, Cotton-Eye Joe?'''

song_input_random_2 = '''So close, no matter how far
Couldn't be much more from the heart
Forever trusting who we are
And nothing else matters'''

## Short fragment, to check logreg functionality
song_input_short = 'Very short sentence, but meaningful'

## Weird input, to check functionality against such cases
song_input_weird = '[chorus] I will never !1618278156%#$#$ gasug fsdygsud ufastduas hvzydf7we uzsuds!!!'

## Single words or very long tests, to check MLP functionality 
song_input_one = 'Singleton'

song_input_long = 'apple '*300

## Meaningless words, lacking interpretation
song_input_what = 'I I I I I I'

In [10]:
import unittest
import sys

class TDD_song_genre(unittest.TestCase):
    
    def test_BasicFunctionality(self):
        self.assertEqual(type(GenrePredictor(song_input_random_1)), str)
        self.assertEqual(type(GenrePredictor(song_input_random_2)), str)
        
    def test_Short(self):
        self.assertEqual(type(GenrePredictor(song_input_short)), str)
    
    def test_Weird(self):
        self.assertEqual(type(GenrePredictor(song_input_weird)), str)
        
    def test_MLP(self):
        self.assertEqual(type(GenrePredictor(song_input_one)), str)
        self.assertEqual(GenrePredictor(song_input_one), GenrePredictor(song_input_one, Neural_only=True))
        
        self.assertEqual(type(GenrePredictor(song_input_long)), str)
        self.assertEqual(GenrePredictor(song_input_long), GenrePredictor(song_input_long, Neural_only=True))
    
    def test_Meaningless(self):
        self.assertEqual(GenrePredictor(song_input_what), "Please input MEANINGFUL lyrics. As far as I'm concerned, that's not really a song.")

In [11]:
if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_BasicFunctionality (__main__.TDD_song_genre) ... ok
test_MLP (__main__.TDD_song_genre) ... ok
test_Meaningless (__main__.TDD_song_genre) ... ok
test_Short (__main__.TDD_song_genre) ... ok
test_Weird (__main__.TDD_song_genre) ... ok

----------------------------------------------------------------------
Ran 5 tests in 84.342s

OK


### 2. Results

#### 2.1 General results

**Snoop Dogg - Doggy Dogg World**

In [89]:
GenrePredictor("We'd like to welcome y'all to the fabulous Carolina West \
I own this motherfucker and my name is Taa-Dow \
Y'all niggaz know who I am y'all niggaz tearin up shit \
But we got somethin old, and somethin new for y'all tongiht \
Put your hands together for Snoop Doggy Dogg \
The Dogg Pound, and the fabulous Dramatics \
[Verse One: Snoop Doggy Dogg] \
It's like everywhere I look, and everywhere I go \
I'm hearin motherfuckers tryin to steal my flow \
But it ain't no thang cause see my nigga Coolio \
Put me up on the game when I step through the do' \
Ya know, some of these niggaz is so deceptive \
Usin my styles like a contraceptive \
I hope ya get burnt, it seems ya havn't learnt \
It's the nick nack patty wack, I still got the bigger sack \
So put your gun away, run away, cuz i'm back (why?) \
Hit em up, get em up, spit em up, now \
Tell me what's goin on \
It make me wanna holler, cuz my dollars come in ozones \
Lone for the break-up, so take off your clothes \
and quit tryin to spit at my motherfuckin hoes \
Seakin of hoes, I'll get to the point \
You think you got the bomb cuz I rolled you a joint \
You'se a flea and i'm the big Dogg \
I'll scratch you off my balls with my motherfuckin paws \
Y'alls, niggaz, better recognize \
And see where I'm comin from it's still East Side till I die \
Why ask why? As the world keeps spinning to the D-O-Double-G-Y \
[Chorus:] \
It's a crazy mixed up world, it's a Doggy Dogg World \
It's a Doggy Dogg World, it's a Doggy Dogg World \
The Dogg's World \
[Verse Two: Kurupt] \
Well if you give me ten bitches then I'll fuck all ten \
Styles vary, packin flavor like Life Savors \
Ain't that somethin, talk shit and I'm dumpin \
I had your whole fuckin block bumpin \
Don't sweat, but check the tecnique, I'm unique like China \
Ya never find the bomb-a-rama then this Nigga behind ya \
So peek-a-boo, clear the way, I'm coming through \
One-two, three, you can't see me \
I'm a G like that strapped with hit hard tactics \
A fuckin menace, usin hoes like tennis rackets \
It's on again, it's on and poppin \
All I see is green, so there ain't no stoppin \
I wanna see some panties droppin \
I'm comin from L.A., she used to chill with Dre up in Compton \
(All I ever did was just use that hoe \
Show her my dickies, get with these, and kick flows)", Neural_only=True)

'Hip Hop'

**Porcupine Tree - Nine Cats**

In [205]:
GenrePredictor("A butterfly sailed on the breeze \
Past a field of barbed wire trees \
Where golden dragons chased around \
Pampered poppies on the ground \
Two silver trout sat way on high \
And watched a royal samurai \
Plant two black orchids in a box \
And strap it to a laughing fox \
A minstrel bought a crooked spoon \
He gave it to a blue baboon \
Who filled it full of virgin snow \
And wandered in the afterglow \
A toad stood in his ballet shoes \
Teaching sixteen kangaroos \
How to skip across a lake \
They found it hard to stay awake \
A pharaoh played a merry tune \
And watched nine cats dance on the moon \
I didn't know what all this meant \
I didn't know why I was sent \
The Insignificance version of Nine Cats has the \
Following additional verse at the end \
I threw 5 clocks down on my bed \
The chimes danced out on golden threads \
And turned to footprints on my wall \
Sequined tears began to fall \
")

'Rock'

**Lady Gaga - Alejandro**

In [209]:
GenrePredictor("I know that we are young \
And I know that you may love me \
But I just can't be with you like this anymore \
Alejandro \
She's got both hands \
In her pocket \
And she won't look at you \
Won't look at you \
She hides true love \
En su bolsillo \
She got a halo around her finger \
Around you \
You know that I love you boy \
Hot like Mexico, rejoice \
At this point I gotta choose \
Nothing to lose \
Don't call my name \
Don't call my name, Alejandro \
I'm not your babe \
I'm not your babe, Fernando \
Don't wanna kiss, don't wanna touch \
Just smoke my cigarette and hush \
Don't call my name \
Don't call my name, Roberto \
Alejandro \
Alejandro \
Ale-ale-jandro \
Ale-ale-jandro \
Alejandro \
Alejandro \
Ale-ale-jandro \
Ale-ale-jandro \
Stop, please \
Just let me go Alejandro \
Just let me go \
She's not broken \
She's just a baby \
But her boyfriend's like a dad, just like a dad \
And all those flames that burned before him \
Now he's gonna fire fight, gotta cool the bad \
You know that I love you boy \
Hot like Mexico, rejoice \
At this point I've gotta choose \
Nothing to lose \
Don't call my name \
Don't call my name, Alejandro \
I'm not your babe \
I'm not your babe, Fernando \
Don't wanna kiss, don't wanna touch \
Just smoke my cigarette and hush \
Don't call my name \
Don't call my name, Roberto \
Alejandro \
Alejandro \
Ale-ale-jandro \
Ale-ale-jandro \
Alejandro \
Alejandro \
Ale-ale-jandro \
Ale-ale-jandro \
Don't bother me \
Don't bother me, Alejandro \
Don't call my name \
Don't call my name, bye, Fernando \
I'm not your babe \
I'm not your babe, Alejandro \
Don't wanna kiss, don't wanna touch \
Fernando \
Don't call my name \
Don't call my name, Alejandro \
I'm not your babe \
I'm not your babe, Fernando \
Don't wanna kiss, don't wanna touch \
Just smoke my cigarette and hush \
Don't call my name \
Don't call my name \
Roberto \
Alejandro \
Alejandro \
Ale-ale-jandro \
Ale-ale-jandro \
Alejandro \
Alejandro \
Ale-ale-jandro \
Ale-ale-jandro \
Don't call my name \
Don't call my name, Alejandro \
I'm not your babe \
I'm not your babe, Fernando \
Don't wanna kiss, don't wanna touch \
Just smoke my cigarette and hush \
Don't call my name \
Don't call my name, Roberto \
Alejandro")

'Pop'

##### METAL

**Dream Theater - On the backs of angels**

In [210]:
GenrePredictor("Standing on the backs of angels \
Destined to create \
Mounting the attack \
While heroes carry your weight \
We spiral towards disaster \
Survival fading faster \
Riding out the wave \
Content to feed off the machine \
Bleeding us to death \
The new American dream \
You're blinded by your hunger \
Beware your days are numbered \
Tears fall from the shameless \
Shelter me, guide me to the edge of the water \
Selfless are the righteous \
Burden me, lead me like a lamb to the slaughter \
Blurring lines drawn in between \
What is right and what is wrong \
Victims on the radar string us along \
We're on to your agenda \
The dead end road to nowhere \
Tears fall from the shameless \
Shelter me, guide me to the edge of the water \
Selfless are the righteous \
Burden me, lead me like a lamb to the slaughter \
Tears fall from the shameless \
Shelter me, guide me to the edge of the water \
Selfless are the righteous \
Burden me, lead me like a lamb to the slaughter \
Selfless are the righteous \
Burden me, lead me like a lamb to the slaughter")

'Rock'

##### RAP

**Eminem - Rap God**

In [211]:
GenrePredictor('''Look, I was gonna go easy on you
Not to hurt your feelings
But I'm only going to get this one chance
Something's wrong, I can feel it
(Six minutes)
Just a feeling I've got
Like something's about to happen
But I don't know what
If that means what I think it means
We're in trouble, big trouble
And if he is as bananas as you say
I'm not taking any chances
You are just what the doc ordered
I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now, who thinks their arms are long enough
To slap box, slap box?
They said I rap like a robot, so call me Rap-bot
But for me to rap like a computer
It must be in my genes
I got a laptop in my back pocket
My pen'll go off when I half-cock it
Got a fat knot from that rap profit
Made a livin' and a killin' off it
Ever since Bill Clinton was still in office
With Monica Lewinsky feelin' on his nutsack
I'm an MC still as honest
But as rude and as indecent as all hell
Syllables, skill-a-holic (Kill 'em all with)
This flippity dippity-hippity hip-hop
You don't really wanna get into a pissin' match
With this rappity brat, packin' a MAC
In the back of the Ac'
Backpack rap crap, yap-yap, yackety-yack
And at the exact same time
I attempt these lyrical acrobat stunts
While I'm practicin' that
I'll still be able to break a motherfuckin' table
Over the back of a couple of faggots and crack it in half
Only realized it was ironic
I was signed to Aftermath after the fact
How could I not blow?
All I do is drop F-bombs
Feel my wrath of attack
Rappers are havin' a rough time period
Here's a maxi pad
It's actually disastrously bad for the wack
While I'm masterfully constructing this masterpiece as
'Cause I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now, who thinks their arms are long enough to slap box, slap box?
Let me show you maintainin' this shit ain't that hard, that hard
Everybody wants the key and the secret
To rap immortality like Ι have got
Well, to be truthful the blueprint's
Simply rage and youthful exuberance
Everybody loves to root for a nuisance
Hit the Earth like an asteroid
Did nothing but shoot for the Moon since (Pew!)
MCs get taken to school with this music
'Cause I use it as a vehicle to "bus the rhyme"
Now I lead a new school full of students
Me? I'm a product of Rakim
Lakim Shabazz, 2Pac, N.W.A, Cube, hey Doc, Ren
Yella, Eazy, thank you, they got Slim
Inspired enough to one day grow up
Blow up and be in a position
To meet Run–D.M.C., induct them
Into the motherfuckin' Rock and Roll Hall of Fame
Even though I'll walk in the church
And burst in a ball of flames
Only Hall of Fame I'll be inducted in
Is the alcohol of fame
On the wall of shame
You fags think it's all a game
'Til I walk a flock of flames
Off a plank and, tell me what in the fuck are you thinkin'?
Little gay-lookin' boy
So gay I can barely say it with a straight face
Lookin' boy (Ha-ha!)
You're witnessin' a massacre
Like you're watching a church gathering take place, lookin' boy
"Oy vey, that boy's gay!"—that's all they say, lookin' boy
You get a thumbs up, pat on the back
And a "way to go" from your label every day, lookin' boy
Hey, lookin' boy! What you say, lookin' boy?
I get a "hell yeah" from Dre, lookin' boy
I'ma work for everything I have
Never asked nobody for shit
Get outta my face, lookin' boy!
Basically, boy, you're never gonna be capable
Of keepin' up with the same pace, lookin' boy, 'cause—
I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
The way I'm racin' around the track
Call me NASCAR, NASCAR
Dale Earnhardt of the trailer park, the White Trash God
Kneel before General Zod
This planet's Krypton—no, Asgard, Asgard
So you'll be Thor, I'll be Odin
You rodent, I'm omnipotent
Let off, then I'm reloadin'
Immediately with these bombs, I'm totin'
And I should not be woken
I'm the walkin' dead, but I'm just a talkin' head
A zombie floatin'
But I got your mom deep-throatin'
I'm out my Ramen Noodle
We have nothin' in common, poodle
I'm a Doberman, pinch yourself in the arm
And pay homage, pupil
It's me, my honesty's brutal
But it's honestly futile if I don't
Utilize what I do though
For good at least once in a while
So I wanna make sure somewhere in this
Chicken scratch I scribble and doodle enough rhymes
To maybe try to help get some people through tough times
But I gotta keep a few punchlines
Just in case, 'cause even you unsigned
Rappers are hungry lookin' at me like it's lunchtime
I know there was a time where once I
Was king of the underground
But I still rap like I'm on my Pharoah Monch grind
So I crunch rhymes, but sometimes when you combine
Appeal with the skin color of mine
You get too big and here they come tryin'
To censor you like that one line
I said on "I'm Back" from The Mathers LP 1 when I
Tried to say I'll take seven kids from Columbine
Put 'em all in a line, add an AK-47, a revolver and a 9
See if I get away with it now
That I ain't as big as I was, but I'm
Morphin' into an immortal, comin' through the portal
You're stuck in a time warp from 2004 though
And I don't know what the fuck that you rhyme for
You're pointless as Rapunzel with fuckin' cornrows
You write normal? Fuck being normal!
And I just bought a new raygun from the future
Just to come and shoot ya, like when Fabolous made Ray J mad
'Cause Fab said he looked like a fag at Mayweather's pad
Singin' to a man while he played piano
Man, oh man, that was a 24-7 special on the cable channel
So Ray J went straight to the radio station
The very next day, "Hey Fab, I'ma kill you!"
Lyrics comin' at you at supersonic speed (J.J. Fad)
Uh, summa-lumma, dooma-lumma, you assumin' I'm a human
What I gotta do to get it through to you I'm superhuman?
Innovative and I'm made of rubber
So that anything you say is ricochetin' off of me
And it'll glue to you and
I'm devastating, more than ever demonstrating
How to give a motherfuckin' audience
A feeling like it's levitating
Never fading, and I know the haters are forever waiting
For the day that they can say I fell off, they'll be celebrating
'Cause I know the way to get 'em motivated
I make elevating music, you make elevator music
"Oh, he's too mainstream."
Well, that's what they do when they get jealous
They confuse it
"It's not hip-hop, it's pop, "—'cause I found a hella way to fuse it
With rock, shock rap with Doc
Throw on "Lose Yourself" and make 'em lose it
"I don't know how to make songs like that
I don't know what words to use."
Let me know when it occurs to you
While I'm rippin' any one of these
S that versus you
It's curtains, I'm inadvertently hurtin' you
How many verses I gotta murder to
Prove that if you were half as nice
Your songs you could sacrifice virgins too?
Ugh, school flunky, pill junkie
But look at the accolades these skills brung me
Full of myself, but still hungry
I bully myself 'cause I make me do
What I put my mind to
And I'm a million leagues above you
Ill when I speak in tongues
But it's still tongue-in-cheek fuck you
I'm drunk, so, Satan, take the fucking wheel
I'ma sleep in the front seat
Bumpin' Heavy D and the Boyz
Still "Chunky but Funky"
But in my head there's something
I can feel tugging and struggling
Angels fight with devils
And here's what they want from me
They're askin' me to eliminate some of the women hate
But if you take into consideration the bitter hatred
I have, then you may be a little patient
And more sympathetic to the situation
And understand the discrimination
But fuck it, life's handin' you lemons?
Make lemonade then!
But if I can't batter the women
How the fuck am I supposed to bake them a cake then?
Don't mistake him for Satan; it's a fatal mistake
If you think I need to be overseas and take a vacation
To trip a broad, and make her fall on her face and
Don't be a retard — Be a king? Think not
Why be a king when you can be a God?''')

'Hip Hop'

##### SPANISH POP

**Luis Fonsi - Despacito** [Translated with google translate](https://translate.google.com/?sl=fr&tl=en&op=translate&hl=en)

In [212]:
GenrePredictor('''Oh!
Fonsi, DY
Oh, oh no, oh no (oh)
Hey yeah
Diridiri, dirididi Daddy
Go!
Yes, you know I've been looking at you for a while
I have to dance with you today (DY)
I saw that your look was already calling me
Show me the way that I'm going
Oh you, you are the magnet and I am the metal
I'm getting closer and I'm putting together the plan
Just thinking about it makes your pulse race (oh yeah)
Ya, I'm already liking you more than normal
All my senses are asking for more
This must be taken without any haste
Slowly
I want to breathe your neck slowly
Let me tell you things in my ear
So that you remember if you are not with me
Slowly
I want to undress you with kisses slowly
Sign the walls of your labyrinth
And make your body a whole manuscript (go up, go up, go up)
(Get up, get up) oh
I want to see your hair dance
I want to be your rhythm (uh oh, uh oh)
That you teach my mouth (uh oh, uh oh)
Your favorite places (favorites, favorites baby)
Let me bypass your danger zones (uh oh, uh oh)
Until I provoke your screams (uh oh, uh oh)
And forget your last name (dirididi Daddy)
I know you're thinking about it (eh)
I've been trying for a long time (eh)
Mommy, this is giving and giving
You know that your heart with me makes you bam bam
You know that that baby is looking for my bam bam
Come test my mouth to see how it tastes to you
I want, I want, I want to see how much love fits you
I'm not in a hurry, I want to take the trip
We start 'slow, then wild
Step by step, soft soft
We are going to stick a little bit
When you kiss me with that skill
I see that you are malice with delicacy
Step by step, soft soft
We are hitting each other, little by little (oh oh)
And that beauty is a puzzle (oh no)
But to assemble it here I have the piece (slow, oh yeah)
Slowly (yeh, me)
I want to breathe your neck slowly (I)
Let me say things in your ear (me)
So that you remember if you are not with me
Slowly
I want to undress you with kisses slowly (yeh)
Sign the walls of your labyrinth
And make your body a whole manuscript (go up, go up, go up)
(Get up, get up) oh
I want to see your hair dance
I want to be your rhythm (uh oh, uh oh)
That you teach my mouth (uh oh, uh oh)
Your favorite places (favorites, favorites baby)
Let me bypass your danger zones (uh oh, uh oh)
Until I provoke your screams (uh oh, uh oh)
And forget your last name
Slowly
Let's do it on a beach in Puerto Rico
Until the waves scream "Ay, blessed"
So that my seal stays with you (dance it)''')

'Pop'

## 2.2 - YOUR predictions

#### AC/DC - The Jack   (Rock)

In [13]:
GenrePredictor('''She gave me the queen
She gave me the king
She was wheelin' and dealin'
Just doin' her thing
''')

'Rock'

#### Porcupine Tree - Radioactive Toy   (Rock)

In [14]:
GenrePredictor('''Feel the sun burning through your black skin
Pour me into a hole, inform my next of kin
Give me the freedom to destroy
Give me radioactive toy
''')

'Rock'

#### Amine - Burden   (Rap/Hip Hop)

In [ ]:
# I'm in limbo but I know the ropes
# And it's really hard to cope with a
# Pilot Jones, when the love is lost, like I'm Coppola
# Translatin' losses before Twitter go and soak it up

In [15]:
GenrePredictor('''This is, this is that shit that you like
How you wanna say this right?
This is, this is like some shit you go and pick your
Your homie from jail with
Look
Uh, uh-oh (yeah, I like that), yeah
Uh, uh-oh (yup)
Bury me before I'm a burden
Don't bury me 'til niggas is certain
I'm singin', singin'
Uh, uh-oh (yeah)
Uh, uh-oh
Bury me before I'm a burden
Don't bury me 'til niggas is certain (yeah)
When your skin darker, shit gets harder (true)
This a black album, like Shawn Carter
Screamin' like I'm week old (yeah)
Moshin' like I'm Steve-O (yeah)
Niggas dodgin' bullets that we jammin' off of pico
At the barbershop, I got the bald fade
That was before the chubby niggas had the Balmain
You know Giuseppe-ass niggas with the Louis shades
Type to call a girl a bitch if she don't give 'em play (yeah)
At sixteen I was survivin' off of noodle cups (damn)
Shit is legal now the dealers know the jig is up
When it's us, niggas gettin' years maybe ten and up
Soccer moms do the same but government don't give a fuck (true)
You know I'm hailin' from that Rip City, get busy
If I die, roll out the yellow carpet if you fuck with me
Yellow roses in the sky (yeah)
Pastor rock a yellow tie, choir make my mama cry
Yellow suit, yeah, I'm fly
Coldplay playin' Yellow live
Uh, uh-oh (goddamn, we gotta make it better), yeah
Uh, uh-oh (if they things don't change)
Bury me before I'm a burden (they stay the same)
Don't bury me 'til niggas is certain
I'm singin', singin' (this shit is so hard)
Uh, uh-oh (we gotta make it better)
Uh, uh-oh (if things don't change)
Bury me before I'm a burden (they stay the same)
Don't bury me 'til niggas is certain (okay)
Man, I'd rather die before I give up
Keep my fucking foot on their necks and never let up
"Mark my words" my nigga Mark used to say that (yeah)
That was way before my nigga Yos had the wave cap (yeah)
And to top it all my best friend had a baby (woo)
We twenty-five, I told that nigga, boy, you fuckin' crazy
Maybe I'm the one who really need the Lord to save me
The last time I went to church was in the fuckin' eighties (yeah)
Can you believe that?
I was born in 94' (woo)
Can you believe that?
Your favorite rapper's doin' blow
Can you believe that?
Wide awake, but comatose
Hollywood's in overdose
Girls send innuendos
I'm in limbo but I know the ropes
And it's really hard to cope with a
Pilot Jones, when the love is lost, like I'm Coppola
Translatin' losses before Twitter go and soak it up
Beats so cold they made Amine wanna open up
Uh, uh-oh (we gotta make it better), yeah
Uh, uh-oh (if they things don't change)
Bury me before I'm a burden (they stay the same)
Don't bury me 'til niggas is certain
I'm singin', singin'
Uh, uh-oh (yeah, I like that)
(We gotta make it better), yeah
Uh, uh-oh (if things don't change, oh things need change)
Bury me before I'm a burden (they stay the same)
Don't bury me 'til niggas is certain (they stay the same, oh)
(Say what?)
Blessings (blessings)
Blessings (blessings)
Blessings (blessings)
Blessings (blessings)
Time's of the essence, I'm tryna get on S's
Man I don't take suggestions
God gave me them
''', Neural_only=True)

'Hip Hop'